# Receive Data (Easy)
This task will create a simple network where the spikes are received from the simulation during the execution.

Set up a function to receive spikes, accepting a label, a time and an array of neuron ids.  Print out the information received.

In [ ]:
def recv_spikes(label, time, neuron_ids):
    print("Received spikes from {} at time {} from neurons {}".format(label, time, neuron_ids))

Set up a ```SpynnakerLiveSpikesConnection``` with a receive label of ```pop``` and local port of None.

In [ ]:
import pyNN.spiNNaker as sim

connection = sim.external_devices.SpynnakerLiveSpikesConnection(local_port=None, receive_labels=["pop"])

Register the function created previously to be called for the label ```pop``` when a spike is received.

In [ ]:
connection.add_receive_callback("pop", recv_spikes)

Setup the simulation with a timestep of 1.0ms.

In [ ]:
sim.setup(1.0)

Create a Population of 100 Poisson neurons at 50Hz and another of 100 LIF neurons, the latter with label ```pop```.  Neither need to be recorded.

In [ ]:
source = sim.Population(100, sim.SpikeSourcePoisson(rate=50), label="source")
pop = sim.Population(100, sim.IF_curr_exp(), label="pop")

Connect the Poisson neurons to the LIF neurons with a fixed probability of 0.1, a weight of 0.5nA and a delay of 1ms.

In [ ]:
sim.Projection(source, pop, sim.FixedProbabilityConnector(0.1), sim.StaticSynapse(weight=0.5, delay=1))

Activate live spike output for the LIF neurons.  Remember to register the connection port.

In [ ]:
sim.external_devices.activate_live_output_for(pop, database_notify_port_num=connection.local_port)

Run for 1000ms and then end the simulation.  Look out for the print from the receive function.

In [ ]:
sim.run(1000)
sim.end()

## Extension 1 (Hard)
Try to plot the spikes as they are received; reduce the rate of the Poisson source to 10Hz.  You can use ```%matplotlib notebook``` to ensure matplotlib is in the correct mode for live drawing.  You may need to only redraw the canvas once every 100 timesteps to work in the notebook.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from threading import RLock

fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()
ax.set_xlim((0, 1000))
ax.set_ylim((0, 100))
fig.show()
fig.canvas.draw()
last_draw = -1
lock = RLock()


def recv_spikes(label, time, neuron_ids):
    global lock
    lock.acquire()
    print("Received spikes from {} at time {} from neurons {}".format(label, time, neuron_ids))
    global ax
    global fig
    global last_draw
    ax.plot([time] * len(neuron_ids), neuron_ids, "b.", markersize=3)
    if time // 100 > last_draw:
        print("Redraw at time {}".format(time))
        fig.canvas.draw()
        last_draw = time // 100
    lock.release()


In [ ]:
import pyNN.spiNNaker as sim

connection = sim.external_devices.SpynnakerLiveSpikesConnection(local_port=None, receive_labels=["pop"])
connection.add_receive_callback("pop", recv_spikes)

sim.setup(1.0)
source = sim.Population(100, sim.SpikeSourcePoisson(rate=20), label="source")
pop = sim.Population(100, sim.IF_curr_exp(), label="pop")
sim.Projection(source, pop, sim.FixedProbabilityConnector(0.1), sim.StaticSynapse(weight=0.5, delay=1))
sim.external_devices.activate_live_output_for(pop, database_notify_port_num=connection.local_port)
sim.run(1000)
sim.end()